In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path

In [2]:
# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PLGA residues and terminal PLGA residue is with 4 Angstroms of BSA, CONFIGURATION 2

Import MDAnalysis

In [5]:
from prot_polymer_analysis import get_protresd_list, aa_frmcount, grptwocnt_aa, gtwo_trjcnt 
from prot_polymer_analysis import frac_cont, bavg_frac_cnt, prot_poly_cntmovie, AA_list_org

In [6]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [7]:
#Units of Angstroms 
dmax = 4.0 

In [8]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 1.2 nm PLGA restrained Rg 100 ns trajectory C2

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system) 

In [9]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb -dump 0 -n bsaplga_nk.ndx 
# to generate a new pdb file that contains unique chain identifiers 
u_pn20_C2 = mda.Universe("../C2_bsa_n20plga_inwater/C2_plgaRes/1.2nm_prodC2/1.2nmC2_bplga.pdb",
                      "../C2_bsa_n20plga_inwater/C2_plgaRes/1.2nm_prodC2/nowat_12nmC2plga.xtc")

In [10]:
u_pn20_C2

<Universe with 10145 atoms>

Check that we are on the first frame

In [11]:
u_pn20_C2.trajectory.frame

0

In [12]:
pn20_lenC2 = len(u_pn20_C2.trajectory)
pn20_lenC2

10001

In [13]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
all_pn20C2 = u_pn20_C2.select_atoms("resname sPLG PLG tPLG and not type H")
all_pn20C2

<AtomGroup with 543 atoms>

In [15]:
# Select BSA residues, heavy atoms only 
protC2 = u_pn20_C2.select_atoms("protein and not type H")
protC2

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [16]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_lenC2 - 1
s_time = timeit.default_timer()
h2di_12nmC2 = aa_frmcount(protC2, all_pn20C2, dmax, u_pn20_C2, start, end)
timeit.default_timer() - s_time

530.191673851572

In [17]:
len(h2di_12nmC2)

182

In [19]:
pr_res_C2 = list(protC2.residues)
ss_res_C2 = [str(row) for row in pr_res_C2]
rkg_C2 = {key:h2di_12nmC2[key][1] for key, value in h2di_12nmC2.items()}
plgC2_1_2nm_occ = pd.DataFrame(data=ss_res_C2, columns=["BSA_des_res"])
plgC2_1_2nm_occ['mda_occ_1.2nmC2'] = plgC2_1_2nm_occ['BSA_des_res'].map(rkg_C2)
plgC2_1_2nm_occ['mda_occ_1.2nmC2'] = plgC2_1_2nm_occ['mda_occ_1.2nmC2'].replace('nan', np.nan).fillna(0)
plgC2_1_2nm_occ['mda_occ_1.2nmC2'] = plgC2_1_2nm_occ['mda_occ_1.2nmC2'].round(2)
plgC2_1_2nm_occ

,BSA_des_res,mda_occ_1.2nmC2
0,"<Residue ASP, 1>",0.42
1,"<Residue THR, 2>",0.40
2,"<Residue HIS, 3>",0.28
3,"<Residue LYS, 4>",0.31
4,"<Residue SER, 5>",0.05
...,...,...
578,"<Residue GLN, 579>",0.00
579,"<Residue THR, 580>",0.00
580,"<Residue ALA, 581>",0.00
581,"<Residue LEU, 582>",0.00


In [20]:
plgC2_1_2nm_occ['mda_occ_1.2nmC2'][plgC2_1_2nm_occ['mda_occ_1.2nmC2'] != 0]

0      0.42
1      0.40
2      0.28
3      0.31
4      0.05
       ... 
541    0.01
543    0.36
544    0.07
546    0.03
547    0.02
Name: mda_occ_1.2nmC2, Length: 113, dtype: float64

In [21]:
bsa_C2 = np.array(list(protC2.resids)) # shape is 4652
mC2_occ_12r = np.array(list(plgC2_1_2nm_occ['mda_occ_1.2nmC2'])) # shape is 583
mC2_occ = np.zeros(shape=(4653))
at_indC2 = np.where(bsa_C2[:-1] != bsa_C2[1:])[0]
at_in_nwC2 = np.sort(np.append([0,4653],at_indC2))
nw_v = 0
for i in range(583):
    b = at_in_nwC2[i+1] +1
    mC2_occ[nw_v:b] = mC2_occ_12r[i]
    nw_v = at_in_nwC2[i+1] + 1 

In [22]:
np.nonzero(mC2_occ)

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
          22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
          33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
          44,   45,   46,   47,   48,   57,   58,   59,   60,   61,   62,
          63,   64,   65,   66,   67,   68,   69,   70,   71,   83,   84,
          85,   86,   87,   88,   89,   90,   91,   92,   93,   94,   95,
          96,   97,   98,   99,  100,  101,  102,  103,  104,  105,  106,
         107,  108,  109,  110,  111,  112,  113,  114,  115,  116,  117,
         118,  119,  120,  121,  122,  123,  124,  125,  126,  127,  128,
         129,  130,  131,  132,  133,  134,  135,  136,  137,  138,  139,
         140,  141,  142,  143,  144,  145,  146,  147,  148,  149,  150,
         389,  390,  391,  392,  393,  394,  395,  396,  397,  414,  415,
         416,  417,  418,  419,  420, 

### Visualize Occupanct on protein 

In [23]:
protC2.occupancies = mC2_occ
protC2.occupancies

array([0.42, 0.42, 0.42, ..., 0.  , 0.  , 0.  ])

In [24]:
with mda.Writer("prot_12nmC2.pdb") as pdb:
    pdb.write(protC2)

### Residue Importance: 1.2 nm restrained C2

In [25]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_12nmC2 = prot_poly_cntmovie(protC2, all_pn20C2, dmax, u_pn20_C2, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [27]:
np.save('1.2nmC2_bplgaRes.npy', trj_ppmap_12nmC2)    # .npy extension is added if not given

In [ ]:
trj_ppmap_12nmC2 = np.load("1.2nmC2_bplgares.npy", allow_pickle=True)

In [28]:
trj_ppmap_12nmC2[0].shape

(583, 60)

In [29]:
np.sum(trj_ppmap_12nmC2[1000][0])

0.0

In [30]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [31]:
pp_12nmC2_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_12nmC2_ct[i][j] = np.sum(trj_ppmap_12nmC2[i][j])

In [33]:
ppC2_12nmtot = np.zeros(shape=(583))
for i in range(583):
    ppC2_12nmtot[i] = np.sum(pp_12nmC2_ct[:,i])
#pp_12nmtot

In [34]:
np.nonzero(ppC2_12nmtot)

(array([  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  19,  20,  40,  44,  47,  48,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  70,  71,  72,  73,  74,  75,  76,  77,  79,  80,  81,
         87,  88,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        119, 120, 123, 125, 126, 127, 129, 130, 133, 153, 154, 156, 157,
        158, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172,
        174, 175, 178, 179, 201, 202, 203, 204, 205, 206, 227, 231, 235,
        238, 241, 245, 247, 250, 257, 260, 261, 263, 264, 266, 267, 271,
        272, 273, 274, 275, 277, 278, 279, 280, 281, 282, 283, 284, 301,
        319, 384, 388, 391, 392, 393, 394, 395, 396, 397, 399, 400, 401,
        403, 404, 407, 408, 433, 437, 502, 516, 519, 538, 539, 540, 541,
        542, 543, 544, 546, 547, 548, 550, 554, 555, 557, 558, 559, 560,
        561, 564, 567, 568, 571, 572, 579, 582]),)

In [35]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, ppC2_12nmtot/np.sum(ppC2_12nmtot), align='center',width=wid, alpha=0.3, color='#1D77CF', label='1.2 nm PLGA')
plt.title("BSA in water with PLGA Restrained Config. 2, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.16])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PLGA contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block, C2

In [36]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [37]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(protC2.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [39]:
pr_res = list(protC2.residues)
ss_res = [str(row) for row in pr_res]
rkg = {key:h2di_12nmC2[key][0] for key, value in h2di_12nmC2.items()}
plgC2_1_2nmaa = pd.DataFrame(data=ss_res, columns=["BSA_des_res"])
plgC2_1_2nmaa['mda_plga_frm_1.2nmC2'] = plgC2_1_2nmaa['BSA_des_res'].map(rkg)
plgC2_1_2nmaa['BSA_des_res'] = red_bsa
plgC2_1_2nmaa['mda_plga_frm_1.2nmC2'] = plgC2_1_2nmaa['mda_plga_frm_1.2nmC2'].replace('nan', np.nan).fillna(0)
plgC2_1_2nmaa.tail()

,BSA_des_res,mda_plga_frm_1.2nmC2
578,GLN 579,0.0
579,THR 580,8.0
580,ALA 581,0.0
581,LEU 582,0.0
582,ALA 583,8.0


In [40]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

pr_res = list(protC2.residues)
ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ss_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [41]:
apl_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_1_2nmaa.iterrows():
    if r_pl['mda_plga_frm_1.2nmC2'] != 0:
        apl_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if r_a['Amino_acids'] in apl_list[i]:
            count += 1
    cpl_l.append(count)      
        
aa_count['plgaC2_1.2nm_100ns'] = cpl_l
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),plgaC2_1.2nm_100ns
0,LYS,59,59,28
1,ARG,23,23,5
2,HIS,16,16,6
3,ASP,40,40,18
4,GLU,59,59,29
5,SER,23,23,4
6,THR,29,29,9
7,CYS,24,24,11
8,TYR,19,19,3
9,ASN,14,14,7


In [42]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plgaC2_1.2nm_100ns'].sum()

182

In [43]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [44]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plgaC2_1.2nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.3737166324435318

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [ ]:
# Mean occupancy and std deviation 
ll_mo = [value[1] for key, value in h2di.items()]
print("Mean Occpancy (1.2 nm Rg): "+str(np.mean(ll_mo)), "Occ. std. dev.: "+str(np.std(ll_mo)))

### Calc. fractional contacts for each AA group type C2

In [45]:
cd_12nmC2 = frac_cont(h2di_12nmC2)
cd_12nmC2

{'Negative': [59978.0, 47, 232255.23404255317, 0.20944569985961872],
 'Positive': [44958.0, 39, 209804.0, 0.1891993771184352],
 'Polar': [55839.0, 42, 241969.0, 0.21820548741668722],
 'Hydrophobic': [53773.0, 54, 181234.92592592593, 0.16343604076800694],
 'Aromatic': [21419.0, 16, 243641.12499999997, 0.21971339483725194]}

In [46]:
cd = frac_cont(h2di_12nmC2)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

182

In [ ]:
start = 0
end = pn20_len - 1
aa_12nmC2, l_f12nmC2 = gtwo_trjcnt(protC2, all_pn20C2, dmax, u_pn20_C2, start, end)

In [ ]:
aa_12nmC2

In [ ]:
l_f12nmC2.shape

In [47]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

487

In [48]:
fcntC2_rg1_2nm, prgrp_1_2nmC2, aaC2_matx = bavg_frac_cnt(5, protC2, all_pn20C2, dmax, u_pn20_C2, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [49]:
fcntC2_rg1_2nm

{'Negative': array([0.18653641, 0.1724291 , 0.16008093, 0.24211444, 0.25282569]),
 'Positive': array([0.17421185, 0.1917324 , 0.17834281, 0.21708371, 0.15567796]),
 'Polar': array([0.35813972, 0.24716185, 0.22217401, 0.19336368, 0.21946825]),
 'Hydrophobic': array([0.15369643, 0.16007837, 0.13285143, 0.15917427, 0.17480614]),
 'Aromatic': array([0.12741558, 0.22859829, 0.30655082, 0.18826391, 0.19722197]),
 'total_frac': array([0.19096509, 0.22176591, 0.20123203, 0.1724846 , 0.14168378])}

In [50]:
prgrp_1_2nmC2

{'Negative': array([array([5.1115    , 3.71497884]), array([9.727     , 4.71003938]),
        array([16.116     ,  3.35880098]), array([21.7665   ,  5.0040961]),
        array([26.7205    ,  6.01850312])], dtype=object),
 'Positive': array([array([3.991     , 4.24498751]), array([10.5175    ,  4.65002083]),
        array([13.475     ,  3.87109481]),
        array([11.9835    ,  2.93755472]), array([9.6905    , 2.66171181])],
       dtype=object),
 'Polar': array([array([8.6415   , 5.5003616]), array([15.0575    ,  4.78332455]),
        array([25.584     ,  4.20249259]),
        array([17.7655    ,  5.79806086]),
        array([18.894     ,  5.34778122])], dtype=object),
 'Hydrophobic': array([array([3.243     , 2.90102585]), array([8.971     , 4.29769229]),
        array([12.536     ,  3.60079214]), array([22.0235   ,  4.6255754]),
        array([25.4915    ,  4.88558367])], dtype=object),
 'Aromatic': array([array([0.819     , 1.51731309]), array([5.6985   , 3.4497243]),
        array

In [51]:
fc_12nmC2_mean = np.array([np.mean(fcntC2_rg1_2nm['Negative']), np.mean(fcntC2_rg1_2nm['Positive'])
                        ,np.mean(fcntC2_rg1_2nm['Polar']),np.mean(fcntC2_rg1_2nm['Hydrophobic'])
                        , np.mean(fcntC2_rg1_2nm['Aromatic'])])
fc_12nmC2_mean

array([0.20279731, 0.18340974, 0.2480615 , 0.15612133, 0.20961011])

In [52]:
fc_12nmC2_std = np.array([np.std(fcntC2_rg1_2nm['Negative']), np.std(fcntC2_rg1_2nm['Positive'])
                       ,np.std(fcntC2_rg1_2nm['Polar']),np.std(fcntC2_rg1_2nm['Hydrophobic'])
                       , np.std(fcntC2_rg1_2nm['Aromatic'])])
fc_12nmC2_std

array([0.03757661, 0.02040848, 0.0576153 , 0.01357548, 0.05852029])

In [53]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc_12nmC2_mean, yerr=fc_12nmC2_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 1.2 nm Rg restrained, Config. 2', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks C2

In [54]:
# Average of total fraction of contacts
np.mean(fcntC2_rg1_2nm['total_frac'])

0.18562628336755646

In [55]:
# Std Deviation of total fraction of contacts
np.std(fcntC2_rg1_2nm['total_frac'])

0.027135816898039913

### Avg no. PLGA residues per BSA AA residue group C2

In [56]:
prgrp_1_2nmC2

{'Negative': array([array([5.1115    , 3.71497884]), array([9.727     , 4.71003938]),
        array([16.116     ,  3.35880098]), array([21.7665   ,  5.0040961]),
        array([26.7205    ,  6.01850312])], dtype=object),
 'Positive': array([array([3.991     , 4.24498751]), array([10.5175    ,  4.65002083]),
        array([13.475     ,  3.87109481]),
        array([11.9835    ,  2.93755472]), array([9.6905    , 2.66171181])],
       dtype=object),
 'Polar': array([array([8.6415   , 5.5003616]), array([15.0575    ,  4.78332455]),
        array([25.584     ,  4.20249259]),
        array([17.7655    ,  5.79806086]),
        array([18.894     ,  5.34778122])], dtype=object),
 'Hydrophobic': array([array([3.243     , 2.90102585]), array([8.971     , 4.29769229]),
        array([12.536     ,  3.60079214]), array([22.0235   ,  4.6255754]),
        array([25.4915    ,  4.88558367])], dtype=object),
 'Aromatic': array([array([0.819     , 1.51731309]), array([5.6985   , 3.4497243]),
        array

In [57]:
mean_12nmC2 = np.zeros(shape=5)
std_12nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_2nmC2.items():
    mpl_12nmC2 = []
    var_12nmC2 = []
    for i in prgrp_1_2nmC2[str(key)].flat:
        mpl_12nmC2.append(i[0])
        var_12nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_12nmC2[count] = np.mean(mpl_12nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_12nmC2[count] = np.std(mpl_12nmC2)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [58]:
mean_12nmC2

array([15.8883,  9.9315, 17.1885, 14.453 ,  6.3684])

In [59]:
std_12nmC2

array([7.82358837, 3.23907745, 5.5012444 , 8.22863616, 3.31655091])

In [60]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_12nmC2, yerr=std_12nmC2, ecolor='black',capsize=5)
plt.title(r'No. of PLGA residues 1.2 nm Rg restrained Config. 2', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PLGA residues')

### Protein/polymer contact map movie C2

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.2 nm res, Config 2", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.5 nm PLGA restrained Rg 100 ns trajectory C2

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [61]:
# Set up the MD Simulation
u15nm_pn20C2 = mda.Universe("../C2_bsa_n20plga_inwater/C2_plgaRes/1.5nm_prodC2/C2_15nmRes.pdb"
                      , "../C2_bsa_n20plga_inwater/C2_plgaRes/1.5nm_prodC2/15nmC2plga_only.xtc")

In [62]:
u15nm_pn20C2

<Universe with 10145 atoms>

In [63]:
pn20_len15C2 = len(u15nm_pn20C2.trajectory)
pn20_len15C2

10001

In [64]:
#Select all the PLGA residues, heavy atoms only 
all15_pn20C2 = u15nm_pn20C2.select_atoms("resname sPLG PLG tPLG and not type H")
all15_pn20C2

<AtomGroup with 543 atoms>

In [65]:
# Select BSA residues, heavy atoms only 
prot_15nmC2 = u15nm_pn20C2.select_atoms("protein and not type H")
prot_15nmC2

<AtomGroup with 4653 atoms>

### Contact Analysis C2

In [66]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len15C2 - 1
s_time = timeit.default_timer()
h2di_15nmC2 = aa_frmcount(prot_15nmC2, all15_pn20C2, dmax, u15nm_pn20C2, start, end)
timeit.default_timer() - s_time
#h2di_15nm

526.3987793847919

In [67]:
len(h2di_15nmC2.keys())

172

In [68]:
pr_res15nmC2 = list(prot_15nmC2.residues)
ss_res15nmC2 = [str(row) for row in pr_res15nmC2]
rkg_15nmC2 = {key:h2di_15nmC2[key][0] for key, value in h2di_15nmC2.items()}
plgC2_1_5nmaa = pd.DataFrame(data=ss_res15nmC2, columns=["BSA_des_res"])
plgC2_1_5nmaa['mda_plga_frm_1.5nmC2'] = plgC2_1_5nmaa['BSA_des_res'].map(rkg_15nmC2)
plgC2_1_5nmaa['BSA_des_res'] = red_bsa
plgC2_1_5nmaa['mda_plga_frm_1.5nmC2'] = plgC2_1_5nmaa['mda_plga_frm_1.5nmC2'].replace('nan', np.nan).fillna(0)
plgC2_1_5nmaa.head()

,BSA_des_res,mda_plga_frm_1.5nmC2
0,ASP 1,5767.0
1,THR 2,5668.0
2,HIS 3,3793.0
3,LYS 4,3736.0
4,SER 5,7.0


In [69]:
# Extract mean occupancy values
pr_res_15nmC2 = list(prot_15nmC2.residues)
ss_res_15nmC2 = [str(row) for row in pr_res_15nmC2]
rkgN_15nmC2 = {key:h2di_15nmC2[key][1] for key, value in h2di_15nmC2.items()}
plgC2_1_2nm_occ['mda_occ_1.5nmC2'] = plgC2_1_2nm_occ['BSA_des_res'].map(rkgN_15nmC2)
plgC2_1_2nm_occ['mda_occ_1.5nmC2'] = plgC2_1_2nm_occ['mda_occ_1.5nmC2'].replace('nan', np.nan).fillna(0)
plgC2_1_2nm_occ['mda_occ_1.5nmC2'] = plgC2_1_2nm_occ['mda_occ_1.5nmC2'].round(2)
plgC2_1_2nm_occ

,BSA_des_res,mda_occ_1.2nmC2,mda_occ_1.5nmC2
0,"<Residue ASP, 1>",0.42,0.58
1,"<Residue THR, 2>",0.40,0.57
2,"<Residue HIS, 3>",0.28,0.38
3,"<Residue LYS, 4>",0.31,0.37
4,"<Residue SER, 5>",0.05,0.00
...,...,...,...
578,"<Residue GLN, 579>",0.00,0.00
579,"<Residue THR, 580>",0.00,0.00
580,"<Residue ALA, 581>",0.00,0.00
581,"<Residue LEU, 582>",0.00,0.00


In [71]:
bsa_15C2 = np.array(list(prot_15nmC2.resids)) # shape is 4652
mC2_occ_15nm = np.array(list(plgC2_1_2nm_occ['mda_occ_1.5nmC2'])) # shape is 583
mC2_occ_15New = np.zeros(shape=(4653))
atC2_ind = np.where(bsa_15C2[:-1] != bsa_15C2[1:])[0]
atC2_in_nw = np.sort(np.append([0,4653],atC2_ind))
nw_v = 0
for i in range(583):
    b = atC2_in_nw[i+1] +1
    mC2_occ_15New[nw_v:b] = mC2_occ_15nm[i]
    nw_v = atC2_in_nw[i+1] + 1 

In [ ]:
mC2_occ_15New[0:33]

In [ ]:
prot_15nmC2.occupancies[3089:3099]

In [ ]:
list(prot_15nmC2.atoms[3089:3099])

In [72]:
np.nonzero(mC2_occ_15New)

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
          22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
          33,   57,   58,   59,   60,   61,   62,   63,   64,   65,   66,
          67,   68,   69,   70,   71,   83,   84,   85,   86,   87,   88,
          89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
         100,  101,  102,  103,  104,  105,  106,  107,  108,  109,  110,
         111,  112,  113,  114,  115,  116,  117,  118,  119,  120,  121,
         122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
         133,  134,  135,  136,  137,  138,  139,  140,  141,  142,  143,
         144,  145,  146,  147,  148,  149,  150,  414,  415,  416,  417,
         418,  419,  420,  421,  422,  430,  431,  432,  433,  434,  435,
         436,  437,  438,  439,  440,  441,  442,  443,  444,  445,  446,
         447,  448,  449,  450,  451, 

### Visualize Occupanct on protein C2

In [74]:
prot_15nmC2.occupancies = mC2_occ_15New
prot_15nmC2.occupancies

array([0.58, 0.58, 0.58, ..., 0.  , 0.  , 0.  ])

In [75]:
with mda.Writer("prot_15nmC2.pdb") as pdb:
    pdb.write(prot_15nmC2)

### Residue Importance: 1.5 nm restrained C2

In [76]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_15nmC2 = prot_poly_cntmovie(prot_15nmC2, all15_pn20C2, dmax, u15nm_pn20C2, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
trj_ppmap_15nmC2 = np.load("1.5nmC2_res.npy", allow_pickle=True)

In [77]:
np.save('1.5nmC2_res.npy', trj_ppmap_15nmC2)    # .npy extension is added if not given

In [78]:
trj_ppmap_15nmC2[0].shape

(583, 60)

In [79]:
np.sum(trj_ppmap_15nmC2[1000][0])

0.0

In [80]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [81]:
ppC2_15nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_15nm_ct[i][j] = np.sum(trj_ppmap_15nmC2[i][j])

In [82]:
ppC2_15nmtot = np.zeros(shape=(583))
for i in range(583):
    ppC2_15nmtot[i] = np.sum(ppC2_15nm_ct[:,i])
#pp_12nmtot

In [83]:
np.nonzero(ppC2_15nmtot)

(array([  0,   1,   2,   3,   4,   5,   7,   8,  10,  11,  12,  13,  14,
         15,  16,  17,  19,  50,  52,  53,  54,  55,  56,  57, 126, 128,
        129, 130, 133, 153, 157, 160, 161, 164, 165, 167, 168, 169, 170,
        171, 175, 178, 179, 182, 186, 231, 238, 256, 257, 259, 260, 261,
        263, 264, 266, 267, 273, 274, 275, 276, 277, 278, 279, 280, 281,
        282, 283, 284, 296, 297, 298, 299, 300, 301, 302, 303, 304, 308,
        309, 314, 315, 318, 319, 321, 332, 335, 336, 337, 338, 339, 350,
        352, 353, 355, 356, 357, 358, 359, 360, 361, 362, 364, 367, 369,
        370, 371, 372, 373, 374, 376, 377, 378, 379, 380, 381, 382, 383,
        384, 385, 386, 387, 388, 390, 391, 392, 393, 394, 395, 396, 404,
        408, 410, 411, 412, 414, 433, 434, 436, 437, 438, 439, 440, 441,
        442, 443, 444, 445, 469, 470, 473, 477, 481, 487, 488, 489, 490,
        491, 492, 493, 494, 535, 536, 537, 538, 539, 540]),)

In [104]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, ppC2_15nmtot/np.sum(ppC2_15nmtot), align='center',width=wid, color='#562A8B', alpha=0.3, label='1.5 nm PLGA')
#plt.bar(y_pos, pp_12nmtot, align='center',width=wid, alpha=0.5, color='#1D77CF',label='1.2 nm PLGA')
plt.title("BSA in water with PLGA Restrained Config. 2, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.16])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PLGA contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block, C2

In [85]:
apl_15nmC2 = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_1_5nmaa.iterrows():
    if r_pl['mda_plga_frm_1.5nmC2'] != 0:
        apl_15nmC2.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_15nmC2 = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_15nmC2)):
        if r_a['Amino_acids'] in apl_15nmC2[i]:
            count += 1
    cpl_15nmC2.append(count)      
        
aa_count['plgaC2_1.5nm_100ns'] = cpl_15nmC2
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),plgaC2_1.2nm_100ns,plgaC2_1.5nm_100ns
0,LYS,59,59,28,28
1,ARG,23,23,5,6
2,HIS,16,16,6,5
3,ASP,40,40,18,15
4,GLU,59,59,29,26
5,SER,23,23,4,5
6,THR,29,29,9,10
7,CYS,24,24,11,12
8,TYR,19,19,3,3
9,ASN,14,14,7,5


In [86]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plgaC2_1.5nm_100ns'].sum()

172

In [87]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [88]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plgaC2_1.5nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.3531827515400411

In [89]:
# Mean occupancy and std deviation 
ll_mo15 = [value[1] for key, value in h2di_15nmC2.items()]
print("Mean Occpancy (1.5 nm Rg): "+str(np.mean(ll_mo15)), "Occ. std. dev.: "+str(np.std(ll_mo15)))

Mean Occpancy (1.5 nm Rg): 0.13188023255813952 Occ. std. dev.: 0.22880708657089754


In [90]:
cd_15nmC2 = frac_cont(h2di_15nmC2)
cd_15nmC2

{'Negative': [62141.0, 41, 260689.0731707317, 0.18803067103970442],
 'Positive': [63358.0, 39, 279425.0256410257, 0.20154460038360955],
 'Polar': [46026.0, 42, 188487.42857142858, 0.13595283164639052],
 'Hydrophobic': [55309.0, 50, 190262.96, 0.13723349278767175],
 'Aromatic': [32620.0, 12, 467553.3333333333, 0.33723840414262385]}

### Calc. fractional contacts for each AA group type, C2

In [91]:
fcntC2_rg1_5nm, prgrp_1_5nmC2, aa_matx_15nmC2 = bavg_frac_cnt(5, prot_15nmC2, all15_pn20C2, dmax,
                                                        u15nm_pn20C2, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [92]:
fcntC2_rg1_5nm

{'Negative': array([0.18819847, 0.18296725, 0.18726038, 0.22900121, 0.17062291]),
 'Positive': array([0.18378093, 0.18007416, 0.21738424, 0.24873969, 0.21072972]),
 'Polar': array([0.09189699, 0.10329381, 0.16180039, 0.1772572 , 0.19911085]),
 'Hydrophobic': array([0.19646371, 0.23636099, 0.14089467, 0.15278241, 0.14244359]),
 'Aromatic': array([0.3396599 , 0.29730379, 0.29266032, 0.19221949, 0.27709293]),
 'total_frac': array([0.18275154, 0.12320329, 0.12936345, 0.16221766, 0.19917864])}

In [93]:
fc_15nmC2_mean = np.array([np.mean(fcntC2_rg1_5nm['Negative']), np.mean(fcntC2_rg1_5nm['Positive'])
                        ,np.mean(fcntC2_rg1_5nm['Polar']),np.mean(fcntC2_rg1_5nm['Hydrophobic'])
                        , np.mean(fcntC2_rg1_5nm['Aromatic'])])
fc_15nmC2_mean

array([0.19161004, 0.20814175, 0.14667185, 0.17378908, 0.27978729])

In [94]:
fc_15nmC2_std = np.array([np.std(fcntC2_rg1_5nm['Negative']), np.std(fcntC2_rg1_5nm['Positive'])
                       ,np.std(fcntC2_rg1_5nm['Polar']),np.std(fcntC2_rg1_5nm['Hydrophobic'])
                       , np.std(fcntC2_rg1_5nm['Aromatic'])])
fc_15nmC2_std

array([0.01971708, 0.02498621, 0.04194312, 0.03724311, 0.048441  ])

In [95]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc_12nmC2_mean, width, yerr=fc_12nmC2_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_15nmC2_mean, width, yerr=fc_15nmC2_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks, C2

In [96]:
np.mean(fcntC2_rg1_5nm['total_frac'])

0.1593429158110883

In [97]:
np.std(fcntC2_rg1_5nm['total_frac'])

0.029488826400026685

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_15nm[0, 0] != 0)

### Avg no. PLGA residues per BSA AA residue group, C2

In [98]:
prgrp_1_5nmC2

{'Negative': array([array([8.127     , 5.41062575]), array([11.7595    ,  3.49995139]),
        array([15.6785    ,  3.50829556]),
        array([16.9215    ,  3.42700128]), array([21.718    ,  4.1395019])],
       dtype=object),
 'Positive': array([array([5.2815   , 4.1057591]), array([11.551     ,  3.47280852]),
        array([24.7385    ,  4.97665729]),
        array([22.024     ,  5.14241422]),
        array([28.3595    ,  4.74881667])], dtype=object),
 'Polar': array([array([1.645     , 2.14615354]), array([7.0735    , 2.84975398]),
        array([18.9585    ,  5.78314601]),
        array([16.925     ,  5.23339039]),
        array([26.3565    ,  4.79618679])], dtype=object),
 'Hydrophobic': array([array([5.3845    , 4.06394633]), array([12.1175    ,  4.00346022]),
        array([9.2825    , 2.27369605]), array([11.001     ,  3.65061625]),
        array([19.993     ,  4.26215333])], dtype=object),
 'Aromatic': array([array([1.997     , 1.81548644]), array([7.7845    , 2.28736961]),

In [100]:
mean_15nmC2 = np.zeros(shape=5)
std_15nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_5nmC2.items():
    mpl_15nmC2 = []
    var_15nmC2 = []
    for i in prgrp_1_5nmC2[str(key)].flat:
        mpl_15nmC2.append(i[0])
        var_15nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_15nmC2[count] = np.mean(mpl_15nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_15nmC2[count] = np.std(mpl_15nmC2)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [101]:
mean_15nmC2

array([14.8409, 18.3909, 14.1917, 11.5557,  9.7728])

In [102]:
std_15nmC2

array([4.62377451, 8.6198031 , 8.78935859, 4.79713436, 4.73165553])

In [103]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_12nmC2, width, yerr=std_12nmC2, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nmC2, width, yerr=std_15nmC2, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PLGA residues Rg restrained Config. 2', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PLGA residues')

### Protein/polymer contact map movie, C2

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res. Config 2", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 2 nm restrained Rg PLGA 100 ns trajectory C2

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

### Contact Analyis C2

In [105]:
# Set up the MD Simulation
u2nm_pn20C2 = mda.Universe("../C2_bsa_n20plga_inwater/C2_plgaRes/2nm_prodC2/2nmResplga.pdb"
                      , "../C2_bsa_n20plga_inwater/C2_plgaRes/2nm_prodC2/C2_2nmResplga.xtc")

In [106]:
u2nm_pn20C2

<Universe with 10145 atoms>

In [107]:
pn20_len2nmC2 = len(u2nm_pn20C2.trajectory)
pn20_len2nmC2

10001

In [108]:
#Select all the PLGA residues, heavy atoms only 
all2nm_pn20C2 = u2nm_pn20C2.select_atoms("resname sPLG PLG tPLG and not type H")
all2nm_pn20C2

<AtomGroup with 543 atoms>

In [109]:
# Select BSA residues, heavy atoms only 
prot_2nmC2 = u2nm_pn20C2.select_atoms("protein and not type H")
prot_2nmC2

<AtomGroup with 4653 atoms>

In [112]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len2nmC2 - 1
s_time = timeit.default_timer()
h2di_2nmC2 = aa_frmcount(prot_2nmC2, all2nm_pn20C2, dmax, u2nm_pn20C2, start, end)
timeit.default_timer() - s_time
h2di_2nmC2

{'<Residue TYR, 30>': [764.0, 0.0764],
 '<Residue LEU, 31>': [659.0, 0.0659],
 '<Residue GLN, 32>': [1175.0, 0.1175],
 '<Residue GLN, 33>': [1082.0, 0.1082],
 '<Residue ARG, 81>': [264.0, 0.0264],
 '<Residue TYR, 84>': [8.0, 0.0008],
 '<Residue GLY, 85>': [23.0, 0.0023],
 '<Residue ASP, 86>': [4228.0, 0.4228],
 '<Residue MET, 87>': [69.0, 0.0069],
 '<Residue ASP, 89>': [1798.0, 0.1798],
 '<Residue CYS, 90>': [857.0, 0.0857],
 '<Residue GLU, 92>': [408.0, 0.0408],
 '<Residue LYS, 93>': [843.0, 0.0843],
 '<Residue PRO, 96>': [13.0, 0.0013],
 '<Residue GLU, 97>': [647.0, 0.0647],
 '<Residue GLU, 100>': [1325.0, 0.1325],
 '<Residue CYS, 101>': [3121.0, 0.3121],
 '<Residue LEU, 103>': [103.0, 0.0103],
 '<Residue SER, 104>': [6981.0, 0.6981],
 '<Residue HIS, 105>': [6103.0, 0.6103],
 '<Residue LYS, 106>': [1876.0, 0.1876],
 '<Residue ASP, 107>': [2283.0, 0.2283],
 '<Residue SER, 109>': [160.0, 0.016],
 '<Residue PRO, 110>': [276.0, 0.0276],
 '<Residue ASP, 111>': [3.0, 0.0003],
 '<Residue TY

In [113]:
len(h2di_2nmC2.keys())

138

In [116]:
pr_res2nmC2 = list(prot_2nmC2.residues)
ss_res2nmC2 = [str(row) for row in pr_res2nmC2]
rkgN_2nmC2 = {key:h2di_2nmC2[key][0] for key, value in h2di_2nmC2.items()}
plgC2_2nmaa = pd.DataFrame(data=ss_res2nmC2, columns=["BSA_des_res"])
plgC2_2nmaa['mda_plga_frm_2nmC2'] = plgC2_2nmaa['BSA_des_res'].map(rkgN_2nmC2)
plgC2_2nmaa['BSA_des_res'] = red_bsa
plgC2_2nmaa['mda_plga_frm_2nmC2'] = plgC2_2nmaa['mda_plga_frm_2nmC2'].replace('nan', np.nan).fillna(0)
plgC2_2nmaa.head()

,BSA_des_res,mda_plga_frm_2nmC2
0,ASP 1,0.0
1,THR 2,0.0
2,HIS 3,0.0
3,LYS 4,0.0
4,SER 5,0.0


In [117]:
# Extract mean occupancy values
pr_res_2urC2 = list(prot_2nmC2.residues)
ss_res_2urC2 = [str(row) for row in pr_res_2urC2]
rkg_2urC2 = {key:h2di_2nmC2[key][1] for key, value in h2di_2nmC2.items()}
plgC2_1_2nm_occ['mda_occ_2nmC2'] = plgC2_1_2nm_occ['BSA_des_res'].map(rkg_2urC2)
plgC2_1_2nm_occ['mda_occ_2nmC2'] = plgC2_1_2nm_occ['mda_occ_2nmC2'].replace('nan', np.nan).fillna(0)
plgC2_1_2nm_occ['mda_occ_2nmC2'] = plgC2_1_2nm_occ['mda_occ_2nmC2'].round(2)
plgC2_1_2nm_occ

,BSA_des_res,mda_occ_1.2nmC2,mda_occ_1.5nmC2,mda_occ_2nmC2
0,"<Residue ASP, 1>",0.42,0.58,0.00
1,"<Residue THR, 2>",0.40,0.57,0.00
2,"<Residue HIS, 3>",0.28,0.38,0.00
3,"<Residue LYS, 4>",0.31,0.37,0.00
4,"<Residue SER, 5>",0.05,0.00,0.00
...,...,...,...,...
578,"<Residue GLN, 579>",0.00,0.00,0.46
579,"<Residue THR, 580>",0.00,0.00,0.07
580,"<Residue ALA, 581>",0.00,0.00,0.00
581,"<Residue LEU, 582>",0.00,0.00,0.05


In [118]:
bsa_2C2 = np.array(list(prot_2nmC2.resids)) # shape is 4652
mC2_occ_2nm = np.array(list(plgC2_1_2nm_occ['mda_occ_2nmC2'])) # shape is 583
mC2_occ_2New = np.zeros(shape=(4653))
atC2_ind2 = np.where(bsa_2C2[:-1] != bsa_2C2[1:])[0]
atC2_in_nw2 = np.sort(np.append([0,4653],atC2_ind2))
nw_v = 0
for i in range(583):
    b = atC2_in_nw2[i+1] +1
    mC2_occ_2New[nw_v:b] = mC2_occ_2nm[i]
    nw_v = atC2_in_nw2[i+1] + 1 

In [119]:
np.nonzero(mC2_occ_2New)

(array([ 237,  238,  239,  240,  241,  242,  243,  244,  245,  246,  247,
         248,  249,  250,  251,  252,  253,  254,  255,  256,  257,  258,
         259,  260,  261,  262,  263,  264,  265,  266,  267,  268,  269,
         270,  271,  272,  273,  274,  634,  635,  636,  637,  638,  639,
         640,  641,  642,  643,  644,  677,  678,  679,  680,  681,  682,
         683,  684,  685,  686,  687,  688,  689,  690,  691,  692,  698,
         699,  700,  701,  702,  703,  704,  705,  706,  707,  708,  709,
         710,  711,  718,  719,  720,  721,  722,  723,  724,  725,  726,
         727,  728,  729,  730,  731,  732,  733,  734,  735,  761,  762,
         763,  764,  765,  766,  767,  768,  769,  789,  790,  791,  792,
         793,  794,  795,  796,  797,  798,  799,  800,  801,  802,  803,
         815,  816,  817,  818,  819,  820,  821,  822,  823,  824,  825,
         826,  827,  828,  829,  830,  831,  832,  833,  834,  835,  836,
         837,  838,  839,  840,  841, 

### Visualize Occupanct on protein C2

In [120]:
prot_2nmC2.occupancies = mC2_occ_2New
prot_2nmC2.occupancies

array([0.  , 0.  , 0.  , ..., 0.12, 0.12, 0.12])

In [121]:
with mda.Writer("prot_2nmC2.pdb") as pdb:
    pdb.write(prot_2nmC2)

### Residue Importance: 2 nm restrained C2

In [122]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trj_ppmap_2nmC2 = prot_poly_cntmovie(prot_2nmC2, all2nm_pn20C2, dmax, u2nm_pn20C2, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [123]:
np.save('2nmC2_res.npy', trj_ppmap_2nmC2)    # .npy extension is added if not given

In [ ]:
trj_ppmap_2nmC2 = np.load('2nmC2_res.npy', allow_pickle=True)
trj_ppmap_2nmC2

In [124]:
trj_ppmap_2nmC2[0].shape

(583, 60)

In [125]:
np.sum(trj_ppmap_2nmC2[1000][0])

0.0

In [126]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [127]:
ppC2_2nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_2nm_ct[i][j] = np.sum(trj_ppmap_2nmC2[i][j])

In [128]:
ppC2_2nmtot = np.zeros(shape=(583))
for i in range(583):
    ppC2_2nmtot[i] = np.sum(ppC2_2nm_ct[:,i])
#pp_12nmtot

In [129]:
np.nonzero(ppC2_2nmtot)

(array([ 29,  30,  31,  32,  80,  83,  84,  85,  86,  88,  89,  91,  92,
         95,  96,  99, 100, 102, 103, 104, 105, 106, 108, 109, 110, 146,
        165, 179, 202, 203, 206, 207, 210, 211, 214, 223, 224, 225, 226,
        227, 228, 229, 230, 231, 233, 234, 235, 238, 260, 261, 263, 264,
        265, 266, 267, 268, 269, 271, 274, 275, 277, 278, 284, 294, 295,
        296, 297, 298, 299, 306, 307, 310, 311, 312, 313, 315, 316, 317,
        319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 330, 331, 334,
        335, 415, 462, 463, 464, 465, 466, 467, 468, 469, 472, 476, 477,
        492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 505,
        506, 533, 534, 535, 536, 541, 567, 568, 570, 571, 572, 574, 575,
        577, 578, 579, 581, 582]),)

In [130]:
a = np.sum(ppC2_2nmtot)

In [131]:
b = np.sum(ppC2_15nmtot)

In [132]:
c = np.sum(ppC2_12nmtot)

In [ ]:
plt.close('all')

In [163]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
#plt.bar(y_pos, ppC2_12nmtot/c, align='center',width=wid, alpha=0.5, color='#1D77CF',label='1.2 nm PLGA')
#plt.bar(y_pos+0.25, ppC2_15nmtot/b, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.5 nm PLGA')
plt.bar(y_pos+0.3, ppC2_2nmtot/a, align='center',width=wid, color='#4E4C4D', alpha=0.3, label='2 nm PLGA')
#plt.bar(y_pos+0.25, pp_15nmtot/b, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.5 nm PLGA')
#plt.bar(y_pos, pp_12nmtot, align='center',width=wid, alpha=0.5, color='#1D77CF',label='1.2 nm PLGA')
plt.title("BSA in water with PLGA Restrained Config 2, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.16])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PLGA contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block C2

In [136]:
apl_2nmC2 = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_2nmaa.iterrows():
    if r_pl['mda_plga_frm_2nmC2'] != 0:
        apl_2nmC2.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_2nmC2 = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_2nmC2)):
        if r_a['Amino_acids'] in apl_2nmC2[i]:
            count += 1
    cpl_2nmC2.append(count)      
        
aa_count['plgaC2_2nm_100ns'] = cpl_2nmC2
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),plgaC2_1.2nm_100ns,plgaC2_1.5nm_100ns,plgaC2_2nm_100ns
0,LYS,59,59,28,28,22
1,ARG,23,23,5,6,4
2,HIS,16,16,6,5,3
3,ASP,40,40,18,15,15
4,GLU,59,59,29,26,16
5,SER,23,23,4,5,5
6,THR,29,29,9,10,10
7,CYS,24,24,11,12,4
8,TYR,19,19,3,3,7
9,ASN,14,14,7,5,3


In [137]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['plgaC2_2nm_100ns'].sum()

138

In [138]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [139]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['plgaC2_2nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.28336755646817247

In [141]:
# Mean occupancy and std deviation 
ll_mo2nm = [value[1] for key, value in h2di_2nmC2.items()]
print("Mean Occpancy (2 nm Rg): "+str(np.mean(ll_mo2nm)), "Occ. std. dev.: "+str(np.std(ll_mo2nm)))

Mean Occpancy (2 nm Rg): 0.18955724637681157 Occ. std. dev.: 0.24720217914700152


In [143]:
cd_2nmC2 = frac_cont(h2di_2nmC2)
cd_2nmC2

{'Negative': [48451.0, 31, 215685.09677419355, 0.1518940705568301],
 'Positive': [40582.0, 29, 193114.3448275862, 0.13599884441476645],
 'Polar': [75326.0, 36, 288749.6666666667, 0.20334906258194266],
 'Hydrophobic': [97230.0, 42, 319470.0, 0.22498354984439767],
 'Aromatic': [46719.0, 16, 402951.375, 0.28377447260206307]}

### Calc. fractional contacts for each AA group type C2

In [144]:
fcntC2_rg2nm, prgrp_2nmC2, aa_matx_2nmC2 = bavg_frac_cnt(5, prot_2nmC2, all2nm_pn20C2, dmax, u2nm_pn20C2, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [145]:
fcntC2_rg2nm

{'Negative': array([0.10673302, 0.14820676, 0.16266585, 0.18306078, 0.17608687]),
 'Positive': array([0.01633748, 0.07863128, 0.14113373, 0.16263226, 0.19665312]),
 'Polar': array([0.19344709, 0.21990673, 0.19367753, 0.20862704, 0.1951992 ]),
 'Hydrophobic': array([0.31858088, 0.31069417, 0.25118076, 0.2265906 , 0.22519734]),
 'Aromatic': array([0.36490153, 0.24256106, 0.25134213, 0.21908932, 0.20686346]),
 'total_frac': array([0.1026694 , 0.11704312, 0.19096509, 0.19917864, 0.18069815])}

In [146]:
fc_2nmC2_mean = np.array([np.mean(fcntC2_rg2nm['Negative']), np.mean(fcntC2_rg2nm['Positive'])
                        ,np.mean(fcntC2_rg2nm['Polar']),np.mean(fcntC2_rg2nm['Hydrophobic'])
                        , np.mean(fcntC2_rg2nm['Aromatic'])])
fc_2nmC2_mean

array([0.15535066, 0.11907758, 0.20217152, 0.26644875, 0.2569515 ])

In [147]:
fc_2nmC2_std = np.array([np.std(fcntC2_rg2nm['Negative']), np.std(fcntC2_rg2nm['Positive'])
                       ,np.std(fcntC2_rg2nm['Polar']),np.std(fcntC2_rg2nm['Hydrophobic'])
                       , np.std(fcntC2_rg2nm['Aromatic'])])
fc_2nmC2_std

array([0.0270779 , 0.06417613, 0.01051749, 0.04049414, 0.05627453])

In [165]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '700px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc_12nmC2_mean, width, yerr=fc_12nmC2_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_15nmC2_mean, width, yerr=fc_15nmC2_std, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc_2nmC2_mean, width, yerr=fc_2nmC2_std, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts PLGA Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.ylim(0,0.4)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks C2

In [149]:
np.mean(fcntC2_rg2nm['total_frac'])

0.15811088295687886

In [150]:
np.std(fcntC2_rg2nm['total_frac'])

0.04009105498764939

### Avg no. PLGA residues per BSA AA residue group C2

In [151]:
prgrp_2nmC2

{'Negative': array([array([1.11      , 1.78742832]), array([8.6505    , 5.33060501]),
        array([9.2035    , 3.26528525]), array([11.5785    ,  2.98794875]),
        array([15.8425    ,  3.73345601])], dtype=object),
 'Positive': array([array([0.143     , 0.57406533]), array([4.457     , 2.93055473]),
        array([7.864     , 4.30679742]), array([14.3065    ,  4.32892108]),
        array([18.779    ,  5.0766287])], dtype=object),
 'Polar': array([array([2.056     , 2.77882421]), array([16.2275    ,  5.46587081]),
        array([24.267     ,  7.98390324]),
        array([36.261     ,  6.23553358]),
        array([35.7015    ,  5.52724142])], dtype=object),
 'Hydrophobic': array([array([3.429     , 4.61367088]), array([16.528     ,  4.08230523]),
        array([24.2835   ,  9.9100014]), array([42.065     ,  5.91487743]),
        array([47.992     ,  6.77944954])], dtype=object),
 'Aromatic': array([array([2.209    , 3.1031144]), array([10.68      ,  3.34269352]),
        array([14.

In [152]:
mean_2nmC2 = np.zeros(shape=5)
std_2nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrp_2nmC2.items():
    mpl_2nmC2 = []
    var_2nmC2 = []
    for i in prgrp_2nmC2[str(key)].flat:
        mpl_2nmC2.append(i[0])
        var_2nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_2nmC2[count] = np.mean(mpl_2nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_2nmC2[count] = np.std(mpl_2nmC2)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [153]:
mean_2nmC2

array([ 9.277 ,  9.1099, 22.9026, 26.8595, 14.3008])

In [154]:
std_2nmC2

array([ 4.80625605,  6.69482169, 12.83166373, 16.37164491,  7.59653133])

In [155]:
mean_15nmC2

array([14.8409, 18.3909, 14.1917, 11.5557,  9.7728])

In [156]:
std_15nmC2

array([4.62377451, 8.6198031 , 8.78935859, 4.79713436, 4.73165553])

In [161]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, mean_12nmC2, width, yerr=std_12nmC2, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nmC2, width, yerr=std_15nmC2, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_2nmC2, width, yerr=std_2nmC2, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PLGA residues Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PLGA residues')

### Protein/polymer contact map movie C2

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm restrained Config 2", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()